In [3]:
!pip install -q -U kaggle_environments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 13.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

# Стратегии, возвращающие одно и тоже значения каждый ход

In [5]:
%%writefile rock.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def rock(observation, configuration):
    return 0

Writing rock.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
%%writefile paper.py

#strategy with only papers
def paper(observation, configuration):
    return 1

Writing paper.py


In [7]:
%%writefile scissors.py

# strategy with only scissors
def scissors(observation, configuration):
    return 2

Writing scissors.py


# Статистические стратегии

In [8]:
%%writefile copy_opponent.py
import random

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


# Тестовые стратегии из библиотеки

In [64]:
%%writefile reactionary.py
import math
import random
last_react_action = None

def reactionary(observation, configuration):
    def get_score(left_move, right_move):
        # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
        delta = (
            right_move - left_move
            if (left_move + right_move) % 2 == 0
            else left_move - right_move
        )
        return 0 if delta == 0 else math.copysign(1, delta)
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_react_action


Overwriting reactionary.py


In [55]:
%%writefile counter_reactionary.py
import random
import math

last_counter_action = None

def counter_reactionary(observation, configuration):
    def get_score(left_move, right_move):
        # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
        delta = (
            right_move - left_move
            if (left_move + right_move) % 2 == 0
            else left_move - right_move
        )
        return 0 if delta == 0 else math.copysign(1, delta)
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs
    return last_counter_action


Overwriting counter_reactionary.py


In [44]:
%%writefile statistical.py
import random
import math

action_histogram = {}

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs


Writing statistical.py


# Рандомная стратегия

In [27]:
%%writefile random.py
import random

def random_choise(observation, configuration):
  return random.randrange(0, configuration.signs)


Overwriting random.py


# Прочие стратегии

In [70]:
%%writefile keep_strategy.py
import random
import math

my_action = []

def keep_strategy(observation, configuration):
    def get_score(left_move, right_move):
        # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
        delta = (
            right_move - left_move
            if (left_move + right_move) % 2 == 0
            else left_move - right_move
        )
        return 0 if delta == 0 else math.copysign(1, delta)
    global my_action
    if observation.step == 0:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    elif get_score(my_action[-1], observation.lastOpponentAction) == 1:
        answer = my_action[-1]
        my_action.append(answer)
    else:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    return answer


Writing keep_strategy.py


In [78]:
%%writefile throw_paper_if_win.py
import random
import math

my_action = []

def keep_strategy(observation, configuration):
    def get_score(left_move, right_move):
        # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
        delta = (
            right_move - left_move
            if (left_move + right_move) % 2 == 0
            else left_move - right_move
        )
        return 0 if delta == 0 else math.copysign(1, delta)
    global my_action
    if observation.step == 0:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    elif get_score(my_action[-1], observation.lastOpponentAction) == 1:
        answer = 1
        my_action.append(answer)
    else:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    return answer


Writing throw_paper_if_win.py


In [83]:
%%writefile throw_scissors_if_lose.py
import random
import math

my_action = []

def keep_strategy(observation, configuration):
    """
    Keep stategy if win
    """
    def get_score(left_move, right_move):
        # This method exists in this file so it can be consumed from rps.py and agents.py without a circular dependency
        delta = (
            right_move - left_move
            if (left_move + right_move) % 2 == 0
            else left_move - right_move
        )
        return 0 if delta == 0 else math.copysign(1, delta)
    global my_action
    if observation.step == 0:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    elif get_score(my_action[-1], observation.lastOpponentAction) < 1:
        answer = 2
        my_action.append(answer)
    else:
        answer = random.randrange(0, configuration.signs)
        my_action.append(answer)
    return answer


Writing throw_scissors_if_lose.py


In [88]:
%%writefile fixed_statistic.py
import random
import numpy

def fixed_statistic(observation, configuration):
    """
    Strategy based on https://avatars.dzeninfra.ru/get-zen_doc/3985649/pub_5faa46b389ace40d9a449e91_5faa816a9c3dc81f90d0e63e/scale_1200
    """
    all_results = [0]*int(configuration.episodeSteps*0.354)+[1]*int(configuration.episodeSteps*0.296) +\
     [2]*int(configuration.episodeSteps*0.35)
    return random.choice(all_results)


Writing fixed_statistic.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

# Создание окружения


In [33]:
env = make('rps',
           configuration={"episodeSteps": 100})

# Запуск окружения

In [100]:
def run_competitions(agent_1:str, agent_2:str, debug=False) -> str:
    """
    Function for run competition between 2 agents and return
    name of the agent, which win thin competitions

    Args:
        agent_1(str): name of file with first agent
        agent_2(str): name of file with second agent
        debug(bool): default - False. If true: print the
            result of the competition, else doesn't
    Return:
        str with name of winner
    """
    result = env.run([agent_1, agent_2]) #start env
    if debug:
        print(result) #for debuging
    result=result[-1] #find last observation
    agent_1_reward = result[0]['observation']['reward'] #find last reward
    agent_2_reward = result[1]['observation']['reward'] #find last reward
    if agent_1_reward>agent_2_reward: # first win
        return agent_1
    elif agent_1_reward<agent_2_reward: # second win
        return agent_2
    else: #tie
        return None

a = run_competitions('random.py', 'fixed_statistic.py')
print(a)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


random.py


# Создаем турнир

In [102]:
import random

# I can convert this to function, but don't want it)

agents = ['rock.py', 'paper.py', 'scissors.py', 'copy_opponent.py',
          'reactionary.py', 'counter_reactionary.py', 'statistical.py',
          'random.py', 'keep_strategy.py', 'throw_paper_if_win.py',
          'throw_scissors_if_lose.py', 'fixed_statistic.py']
# First round
temp = agents.copy()
tree=[] # tree of championship
winner_tree=[] # list with winners of round
while temp!=[]:
    leave = random.sample(temp, k=2) # random choice not repeated values
    temp.remove(leave[0])
    temp.remove(leave[1])
    tree.append(leave) # add new pair of players
for i in tree: # for all pairs
    winner=None
    while winner is None:
        winner = run_competitions(i[0], i[1]) #start competition
    winner_tree.append(winner)

print(f'Winner of 1 round is: {winner_tree}') #print winners of the first round
# Second round (same things)
temp = winner_tree.copy()
tree=[]
winner_tree=[]
while temp!=[]:
    leave = random.sample(temp, k=2)
    temp.remove(leave[0])
    temp.remove(leave[1])
    tree.append(leave)
for i in tree:
    winner=None
    while winner is None:
        winner = run_competitions(i[0], i[1])
    winner_tree.append(winner)
print(f'Winner of 2 round is: {winner_tree}')
# Third round (change, because number of agents is odd)
winner_1=None
winner_2=None
total_winner = None

while winner_1 is None:
    winner_1 = run_competitions(winner_tree[0], winner_tree[1]) #run first and second agent
while winner_2 is None:
    winner_2 = run_competitions(winner_1, winner_tree[2]) #run winner and second agent
while total_winner is None:
    total_winner = run_competitions(winner_1, winner_2) #run winners
print(f'Total winner is {total_winner}') #WE HAVE A WINNER, CONGRATULATIONS!!!!

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Winner of 1 round is: ['counter_reactionary.py', 'throw_paper_if_win.py', 'fixed_statistic.py', 'reactionary.py', 'copy_opponent.py', 'keep_strategy.py']
Winner of 2 round is: ['copy_opponent.py', 'keep_strategy.py', 'reactionary.py']
Total winner is copy_opponent.py


# Старый код, который требуется переосознать

In [45]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent.py", "test.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

TypeError: '<' not supported between instances of 'str' and 'int'

In [35]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "random.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[0, 0]]